Карпеченко Дмитрий, дз 25 v.1

Наглым образом возьмем предыдущую работу с данным датасетом и оставим всю обработку. Сравним с метриками RandomForestClassifier, полученными раннее

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-darkgrid'])
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import validation_curve, learning_curve
RANDOM_STATE = 17

In [2]:
def make_submission(predictions, fname):
    out = pd.DataFrame(data=predictions, columns=['Probability'])
    out.index += 1
    out.to_csv(fname, index_label='id')

In [3]:
df = pd.read_csv(r'D:\TeachMeSkills\DZ\dz18\cs-training.csv', index_col=0)

In [4]:
feature_names = df.columns[1:]
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [5]:
print(df.shape)
df.describe(include = "all").T

(150000, 11)


,count,mean,std,min,25%,50%,75%,max
SeriousDlqin2yrs,150000.0,0.066840,0.249746,0.0,0.000000,0.000000,0.000000,1.0
RevolvingUtilizationOfUnsecuredLines,150000.0,6.048438,249.755371,0.0,0.029867,0.154181,0.559046,50708.0
age,150000.0,52.295207,14.771866,0.0,41.000000,52.000000,63.000000,109.0
NumberOfTime30-59DaysPastDueNotWorse,150000.0,0.421033,4.192781,0.0,0.000000,0.000000,0.000000,98.0
DebtRatio,150000.0,353.005076,2037.818523,0.0,0.175074,0.366508,0.868254,329664.0
MonthlyIncome,120269.0,6670.221237,14384.674215,0.0,3400.000000,5400.000000,8249.000000,3008750.0
NumberOfOpenCreditLinesAndLoans,150000.0,8.452760,5.145951,0.0,5.000000,8.000000,11.000000,58.0
NumberOfTimes90DaysLate,150000.0,0.265973,4.169304,0.0,0.000000,0.000000,0.000000,98.0
NumberRealEstateLoansOrLines,150000.0,1.018240,1.129771,0.0,0.000000,1.000000,2.000000,54.0
NumberOfTime60-89DaysPastDueNotWorse,150000.0,0.240387,4.155179,0.0,0.000000,0.000000,0.000000,98.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

In [7]:
df.isnull().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

Удалим записи, где возраст равен 0. Такая запись одна)

In [8]:
df=df[(df['age']!=0)]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149999 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149999 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149999 non-null  float64
 2   age                                   149999 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149999 non-null  int64  
 4   DebtRatio                             149999 non-null  float64
 5   MonthlyIncome                         120268 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149999 non-null  int64  
 7   NumberOfTimes90DaysLate               149999 non-null  int64  
 8   NumberRealEstateLoansOrLines          149999 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149999 non-null  int64  
 10  NumberOfDependents                    146075 non-null  float64
dtype

Пропуски в столбце MonthlyIncome заменим медианными значениями

In [10]:
df['MonthlyIncome']=df['MonthlyIncome'].fillna(df['MonthlyIncome'].median())

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149999 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149999 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149999 non-null  float64
 2   age                                   149999 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149999 non-null  int64  
 4   DebtRatio                             149999 non-null  float64
 5   MonthlyIncome                         149999 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149999 non-null  int64  
 7   NumberOfTimes90DaysLate               149999 non-null  int64  
 8   NumberRealEstateLoansOrLines          149999 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149999 non-null  int64  
 10  NumberOfDependents                    146075 non-null  float64
dtype

Пропуски в столбце NumberOfDependents заменим на 0, как самый распростаненный

In [12]:
df['NumberOfDependents']=df['NumberOfDependents'].fillna(0)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149999 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149999 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149999 non-null  float64
 2   age                                   149999 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149999 non-null  int64  
 4   DebtRatio                             149999 non-null  float64
 5   MonthlyIncome                         149999 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149999 non-null  int64  
 7   NumberOfTimes90DaysLate               149999 non-null  int64  
 8   NumberRealEstateLoansOrLines          149999 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149999 non-null  int64  
 10  NumberOfDependents                    149999 non-null  float64
dtype

Для столбца NumberOfTime60-89DaysPastDueNotWorse удалим выбивающиеся значения 98 и 96

In [14]:
df['NumberOfTime60-89DaysPastDueNotWorse'].value_counts()

0     142395
1       5731
2       1118
3        318
98       264
4        105
5         34
6         16
7          9
96         5
8          2
11         1
9          1
Name: NumberOfTime60-89DaysPastDueNotWorse, dtype: int64

In [15]:
df = df.loc[df['NumberOfTime60-89DaysPastDueNotWorse'] != 98]

In [16]:
df = df.loc[df['NumberOfTime60-89DaysPastDueNotWorse'] != 96]

Вместо столбцов NumberOfTime30-59DaysPastDueNotWorse, NumberOfTime60-89DaysPastDueNotWorse, NumberOfTimes90DaysLate введем новый столбец в котором будет среднее количество просрочек за период от 30 до бесконечности дней

In [17]:
t=(df['NumberOfTime60-89DaysPastDueNotWorse']+df['NumberOfTime30-59DaysPastDueNotWorse']+df['NumberOfTimes90DaysLate'])/3
df.insert(2,'MeanPastDueNotWorse',t)
df.drop(labels=['NumberOfTime60-89DaysPastDueNotWorse','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate'], axis=1, inplace=True)

In [18]:
df.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,MeanPastDueNotWorse,age,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,NumberOfDependents
count,149730.000000,149730.000000,149730.000000,149730.000000,149730.000000,1.497300e+05,149730.000000,149730.000000,149730.000000
mean,0.065979,6.057542,0.133690,52.327984,353.631316,6.423097e+03,8.467949,1.020063,0.738142
std,0.248246,249.980364,0.367383,14.754371,2039.601344,1.290112e+04,5.138107,1.129961,1.107373
min,0.000000,0.000000,0.000000,21.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,0.029771,0.000000,41.000000,0.175994,3.915000e+03,5.000000,0.000000,0.000000
50%,0.000000,0.153488,0.000000,52.000000,0.367119,5.400000e+03,8.000000,1.000000,0.000000
75%,0.000000,0.555592,0.000000,63.000000,0.870023,7.400000e+03,11.000000,2.000000,1.000000
max,1.000000,50708.000000,6.333333,109.000000,329664.000000,3.008750e+06,58.000000,54.000000,20.000000


In [19]:
x=df.copy()
y=df.SeriousDlqin2yrs
x.drop(labels=['SeriousDlqin2yrs'],axis=1, inplace=True)

In [20]:
param_grid_rfc = [
     {'rfc__max_features': [3,4],
      'rfc__max_depth': [6,7]}]

In [21]:
param_grid_log = [
     { 'log__penalty': ['l2', 'l1'],
        'log__solver':['liblinear']}]
    

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y,random_state=42)

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipe_rfc = Pipeline([("rfc", RandomForestClassifier())])
pipe_log = Pipeline([("scaler", StandardScaler()), ("log", LogisticRegression())])

RandomForestClassifier

In [24]:
ggrid = GridSearchCV(pipe_rfc, param_grid=param_grid_rfc, cv=5)
ggrid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('rfc', RandomForestClassifier())]),
             param_grid=[{'rfc__max_depth': [6, 7],
                          'rfc__max_features': [3, 4]}])

In [25]:
print("Лучшие параметры RandomForest:\n{}".format(ggrid.best_estimator_))

Лучшие параметры RandomForest:
Pipeline(steps=[('rfc', RandomForestClassifier(max_depth=7, max_features=4))])


In [29]:
from sklearn.metrics import roc_auc_score, roc_curve, auc,accuracy_score
roc_auc_score(y_test, ggrid.predict_proba(X_test)[:, 1])

0.8605559053973177

In [30]:
accuracy_score(y_test, ggrid.predict(X_test))

0.9369005957310395

In [49]:
import tensorflow as tf
from tensorflow import keras

In [50]:
X_train_full, X_test, y_train_full, y_test = train_test_split(x, y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [51]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [62]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Dense(30, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [63]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 8)                 0         
                                                                 
 dense_16 (Dense)            (None, 30)                270       
                                                                 
 dense_17 (Dense)            (None, 100)               3100      
                                                                 
 dense_18 (Dense)            (None, 1)                 101       
                                                                 
Total params: 3,471
Trainable params: 3,471
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [65]:
history=model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), batch_size=128)

Epoch 1/10
658/658 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.9341 - val_loss: nan - val_accuracy: 0.9329
Epoch 2/10
658/658 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.9349 - val_loss: nan - val_accuracy: 0.9329
Epoch 3/10
658/658 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.9349 - val_loss: nan - val_accuracy: 0.9329
Epoch 4/10
658/658 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.9349 - val_loss: nan - val_accuracy: 0.9329
Epoch 5/10
658/658 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.9349 - val_loss: nan - val_accuracy: 0.9329
Epoch 6/10
658/658 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.9349 - val_loss: nan - val_accuracy: 0.9329
Epoch 7/10
658/658 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.9349 - val_loss: nan - val_accuracy: 0.9329
Epoch 8/10
658/658 [============================

In [66]:
model.evaluate(X_test, y_test)

1170/1170 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.9329


[nan, 0.9328666925430298]

30 - 100 -1,  epochs=20, batch_size=32  accuracy =  0.9328
60 - 100 -1,  epochs=20, batch_size=32  accuracy =  0.9328
30-60-100-1,  epochs=20, batch_size=32  accuracy =  0.9328
30 - 100 -1,  epochs=10, batch_size=128  accuracy =  0.9328

Получили метрику точности чуть меньше, чем при реализации классической модели рандомфорест. Количество слов,эпох, нейронов и размер batch_size не смогли никак повлиять на результат